# A Real Dataset

In [5]:
import os
import numpy as np
from pathlib import Path
from config import *

In [ ]:
# Download the data 
if not mnist_name in os.listdir(DATASETS_DIR):

    import urllib
    import urllib.request

    file_name = f'{DATASETS_DIR.joinpath(f"{mnist_name}.zip")}'
    # Dowloading
    print(f'Downloading as {DATASETS_DIR.joinpath(mnist_name)}.zip from {mnist_url}....')
    urllib.request.urlretrieve(mnist_url, file_name, )
    print('Successfully Downloaded')

    #Unzip and delete zip file
    from zipfile import ZipFile

    print(f'Unzipping {mnist_name}.zip....')
    with ZipFile(file_name) as files:
        files.extractall(DATASETS_DIR.joinpath(mnist_name))
    print('Successfully Unzipped')

    print('Deleting zip file....')
    os.remove(file_name)
    print('Done')
else:
    print('Folder already in', DATASETS_DIR)

Successfully Downloaded
Unzipping fashion_mnist_images.zip....
Successfully Unzipped
Deleting zip file....
